In [1]:
import pandas as pd
import time
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
# 数据加载
data = pd.read_csv('Market_Basket_Optimisation.csv',header=None,sep='\t',names=['Transaction'])
data = data.fillna(0)
data['rank'] = data.index

In [3]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [4]:
transaction = pd.DataFrame(columns=['TransactionID','Transaction'])
for i,v in data.iterrows():
    for s in v:
        if type(s) is str:
            for t in s.split(','):
                s1 = pd.Series({'TransactionID' : i,'Transaction' : t})
                transaction=transaction.append(s1,ignore_index=True)

In [5]:
pd.options.display.max_columns = 100
start = time.time()
hot_encoded_df = transaction.groupby(['TransactionID','Transaction'])['Transaction'].count().unstack().reset_index().fillna(0)
hot_encoded_df = hot_encoded_df.applymap(encode_units)
frequent_itemsets = apriori(hot_encoded_df, min_support=0.02, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=0.1)
print("频繁项集: ", frequent_itemsets)
print("关联规则: ", rules[ (rules['lift'] >= 1) & (rules['confidence'] >= 0.5)])
# print(rules['confidence'])
end = time.time()
print("用时: ", end-start)

频繁项集:        support                                          itemsets
0    0.999867                                   (TransactionID)
1    0.020397                                         (almonds)
2    0.033329                                         (avocado)
3    0.033729                                        (brownies)
4    0.087188                                         (burgers)
..        ...                                               ...
202  0.020131  (whole wheat rice, mineral water, TransactionID)
203  0.022930             (olive oil, spaghetti, TransactionID)
204  0.025197              (pancakes, spaghetti, TransactionID)
205  0.021197                (shrimp, spaghetti, TransactionID)
206  0.020931              (tomatoes, spaghetti, TransactionID)

[207 rows x 2 columns]
关联规则:                             antecedents      consequents  antecedent support  \
4                           (brownies)  (TransactionID)            0.033729   
6                            (burger

In [6]:
print(transaction)

TransactionID      Transaction
0                 0           shrimp
1                 0          almonds
2                 0          avocado
3                 0   vegetables mix
4                 0     green grapes
...             ...              ...
29358          7499        green tea
29359          7500             eggs
29360          7500  frozen smoothie
29361          7500      yogurt cake
29362          7500   low fat yogurt

[29363 rows x 2 columns]
